In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_all = pd.read_csv('../data/data.csv')

train, test = train_test_split(data_all, test_size=0.2, random_state=42)

X_train = train.drop(['CREDIT_SCORE','DEFAULT'], axis=1)
y_train = train['DEFAULT']

X_test = test.drop(['CREDIT_SCORE','DEFAULT'], axis=1)
y_test = test['DEFAULT']

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

class BaseColumnsSelector(BaseEstimator, TransformerMixin):
    def __init__(self, subset):
        self.subset = subset
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.loc[:, self.subset]
    
baseColumns = ['INCOME', 'SAVINGS', 'DEBT', 'T_CLOTHING_12', 'T_CLOTHING_6', 
       'T_EDUCATION_12', 'T_EDUCATION_6', 'T_ENTERTAINMENT_12',
       'T_ENTERTAINMENT_6', 'T_FINES_12',
       'T_FINES_6', 'T_GAMBLING_12', 'T_GAMBLING_6', 
       'T_GROCERIES_12', 'T_GROCERIES_6', 'T_HEALTH_12', 'T_HEALTH_6',
       'T_HOUSING_12', 'T_HOUSING_6', 'T_TAX_12', 'T_TAX_6', 'T_TRAVEL_12',
       'T_TRAVEL_6', 'R_TRAVEL', 'T_UTILITIES_12', 'T_UTILITIES_6', 
       'T_EXPENDITURE_12', 'T_EXPENDITURE_6', 
       'CAT_GAMBLING', 'CAT_CREDIT_CARD', 'CAT_MORTGAGE',
       'CAT_SAVINGS_ACCOUNT', 'CAT_DEPENDENTS']

groups = ['CLOTHING', 'EDUCATION', 'ENTERTAINMENT', 'FINES', 'GAMBLING', 'GROCERIES', 'HEALTH', 'HOUSING', 'TAX', 'TRAVEL', 'UTILITIES']
no_sense_in_division =['T_EDUCATION_12', 'T_FINES_12', 'T_GAMBLING_12', 'T_HOUSING_12', 'T_TAX_12', 'T_TRAVEL_12']
division_groups = ['CLOTHING',  'ENTERTAINMENT',  'GROCERIES', 'HEALTH',  'UTILITIES']


In [4]:
#być może zrobić fines_6 i fines_12 jako kategoryczne???? i gambling_6 i gambling_12 też
class OutliersReplacer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for column in X.columns:
            if column == 'SAVINGS':
                X.loc[X[column] > 2500000, column] = 2500000
            elif column == 'DEBT':
                X.loc[X[column] > 4000000, column] = 4000000
            elif column == 'T_CLOTHING_12':
                X.loc[X[column] > 32000, column] = 32000
            elif column == 'T_CLOTHING_6':
                X.loc[X[column] > 25000, column] = 25000
            elif column == 'T_HEALTH_12':
                X.loc[X[column] > 25000, column] = 25000
            elif column == 'T_HEALTH_6':
                X.loc[X[column] > 18000, column] = 18000
            elif column == 'T_TRAVEL_12':
                X.loc[X[column] > 150000, column] = 150000
            elif column == 'T_TRAVEL_6':
                X.loc[X[column] > 110000, column] = 110000
        return X

In [5]:
class RatioColumnsAdder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for idx, row in X.iterrows():
            #DEBT/INCOME
            if row['INCOME'] == 0:
                X.at[idx, 'R_DEBT_INCOME'] = None
            else:
                X.at[idx, 'R_DEBT_INCOME'] = row['DEBT'] / row['INCOME']
            #SAVINGS/INCOME
            if row['INCOME'] == 0:
                X.at[idx, 'R_SAVINGS_INCOME'] = None
            else:
                X.at[idx, 'R_SAVINGS_INCOME'] = row['SAVINGS'] / row['INCOME']
            #DEBT/SAVINGS
            if row['SAVINGS'] == 0:
                X.at[idx, 'R_DEBT_SAVINGS'] = None
            else:
                X.at[idx, 'R_DEBT_SAVINGS'] = row['DEBT'] / row['SAVINGS']
            for group in division_groups:
                if row['T_' + group + '_12'] == 0:
                    X.at[idx, 'R_' + group] = None
                else:
                    X.at[idx, 'R_' + group] = row['T_' + group + '_6'] / row['T_' + group + '_12']
            for group in groups:
                if row['INCOME'] == 0:
                    X.at[idx, 'R_'+group+'_INCOME'] = None
                else:
                    X.at[idx, 'R_'+group+'_INCOME'] = row['T_'+group+'_6'] / row['INCOME']
                    
                if row['SAVINGS'] == 0:
                    X.at[idx, 'R_'+group+'_SAVINGS'] = None
                else:
                    X.at[idx, 'R_'+group+'_SAVINGS'] = row['T_'+group+'_6'] / row['SAVINGS']

                if row['DEBT'] == 0:
                    X.at[idx, 'R_'+group+'_DEBT'] = None
                else:
                    X.at[idx, 'R_'+group+'_DEBT'] = row['T_'+group+'_6'] / row['DEBT']
        return X

In [6]:
class DropColumns(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        potentialColumnsToDrop = ['T_EDUCATION_12', 'T_FINES_12', 'T_GAMBLING_12', 'T_HOUSING_12', 'T_TAX_12', 'T_TRAVEL_12', 'T_EDUCATION_6','T_ENTERTAINMENT_6','T_GAMBLING_6','T_GROCERIES_6','T_HOUSING_6','T_EXPENDITURE_12', 'T_EXPENDITURE_6','R_GROCERIES_DEBT', 'INCOME', 'T_UTILITIES_6', 'R_EDUCATION_DEBT', 'T_UTILITIES_12', 'R_CLOTHING_DEBT',
        'CAT_DEPENDENTS', 'R_ENTERTAINMENT_SAVINGS', 'R_FINES_INCOME',
        'R_FINES_SAVINGS', 'R_FINES_DEBT', 'R_GROCERIES_SAVINGS',
        'CAT_SAVINGS_ACCOUNT', 'R_HOUSING_INCOME', 'R_TAX_INCOME',
        'R_TAX_SAVINGS', 'R_TRAVEL_DEBT', 'R_UTILITIES_DEBT', 'CAT_GAMBLING',
        'CAT_DEBT', 'CAT_MORTGAGE', 'SAVINGS', 'R_UTILITIES_SAVINGS', 'R_EDUCATION', 'R_FINES', 'R_GAMBLING', 'R_HOUSING', 'R_GROCERIES_INCOME', 'T_ENTERTAINMENT_12', 'R_ENTERTAINMENT',
       'R_TRAVEL_SAVINGS', 'R_GAMBLING_SAVINGS', 'T_CLOTHING_6']
        for column in potentialColumnsToDrop:
            if column in X.columns:
                X.drop(column, axis=1, inplace=True)
        return X

In [7]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

class MissingValuesFiller(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        numerical_columns = X.select_dtypes(include=[np.number]).columns
        imputer = KNNImputer(n_neighbors=7)
        X[numerical_columns] = imputer.fit_transform(X[numerical_columns])
        return X

In [7]:
#best parameters :
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

for n_est in range(13, 24, 1):
    for depth in range(2, 4, 1):
        for lr in [0.1, 0.2, 0.3]:
            pipeline = Pipeline([
                ('selector', BaseColumnsSelector(baseColumns)),
                ('outliers_replacer', OutliersReplacer()),
                ('ratio_columns_adder', RatioColumnsAdder()),
                ('missing_values_filler', MissingValuesFiller()),
                ('drop_columns', DropColumns()),
                ('classifier', GradientBoostingClassifier(n_estimators=n_est, max_depth=depth, learning_rate=lr))
            ])
            kfold = KFold(n_splits=5, random_state=42, shuffle=True)
            score_acc = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
            score_recall = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='recall')
            score_precision = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='precision')
            score_f1 = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='f1')
            print(f"n_estimators: {n_est}, max_depth: {depth}, learning_rate: {lr}, accuracy: {score_acc.mean()}, recall: {score_recall.mean()}, precision: {score_precision.mean()}, f1: {score_f1.mean()}")


n_estimators: 13, max_depth: 2, learning_rate: 0.1, accuracy: 0.728125, recall: 0.11821511821511824, precision: 0.621904761904762, f1: 0.19643381600519877
n_estimators: 13, max_depth: 2, learning_rate: 0.2, accuracy: 0.728125, recall: 0.16286380286380286, precision: 0.5775824175824175, f1: 0.2497425765048183
n_estimators: 13, max_depth: 2, learning_rate: 0.3, accuracy: 0.715625, recall: 0.17824841824841825, precision: 0.503030303030303, f1: 0.25432700036536354
n_estimators: 13, max_depth: 3, learning_rate: 0.1, accuracy: 0.721875, recall: 0.13546453546453546, precision: 0.6039393939393939, f1: 0.19850202881042112
n_estimators: 13, max_depth: 3, learning_rate: 0.2, accuracy: 0.70625, recall: 0.15690975690975692, precision: 0.4288888888888889, f1: 0.22000750938673339
n_estimators: 13, max_depth: 3, learning_rate: 0.3, accuracy: 0.725, recall: 0.20086580086580086, precision: 0.49401960784313725, f1: 0.3295098039215686
n_estimators: 14, max_depth: 2, learning_rate: 0.1, accuracy: 0.7296875

KeyboardInterrupt: 

In [8]:
#best parameters :
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

for n_est in range(17, 20, 1):
    for depth in range(2, 5, 1):
        for lr in [ 0.1, 0.15, 0.2, 0.25]:
            pipeline = Pipeline([
                ('selector', BaseColumnsSelector(baseColumns)),
                ('outliers_replacer', OutliersReplacer()),
                ('ratio_columns_adder', RatioColumnsAdder()),
                ('missing_values_filler', MissingValuesFiller()),
                ('drop_columns', DropColumns()),
                ('classifier', GradientBoostingClassifier(n_estimators=n_est, max_depth=depth, learning_rate=lr))
            ])
            kfold = KFold(n_splits=5, random_state=42, shuffle=True)
            score_acc = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
            score_recall = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='recall')
            score_precision = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='precision')
            score_f1 = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='f1')
            print(f"n_estimators: {n_est}, max_depth: {depth}, learning_rate: {lr}, accuracy: {score_acc.mean()}, recall: {score_recall.mean()}, precision: {score_precision.mean()}, f1: {score_f1.mean()}")


n_estimators: 17, max_depth: 2, learning_rate: 0.1, accuracy: 0.7296875, recall: 0.12368964368964368, precision: 0.6305194805194805, f1: 0.21217842297240325
n_estimators: 17, max_depth: 2, learning_rate: 0.15, accuracy: 0.734375, recall: 0.16286380286380286, precision: 0.6039393939393939, f1: 0.2555566301310982
n_estimators: 17, max_depth: 2, learning_rate: 0.2, accuracy: 0.7375, recall: 0.17824841824841825, precision: 0.6343589743589744, f1: 0.2764251207729469
n_estimators: 17, max_depth: 2, learning_rate: 0.25, accuracy: 0.7203125, recall: 0.1731202131202131, precision: 0.544141414141414, f1: 0.249606452510082
n_estimators: 17, max_depth: 3, learning_rate: 0.1, accuracy: 0.725, recall: 0.15084915084915082, precision: 0.575, f1: 0.2303522076171217
n_estimators: 17, max_depth: 3, learning_rate: 0.15, accuracy: 0.7234375, recall: 0.1848951048951049, precision: 0.5347652347652347, f1: 0.2681523855919179
n_estimators: 17, max_depth: 3, learning_rate: 0.2, accuracy: 0.7046875, recall: 0.17

KeyboardInterrupt: 

In [9]:
#best parameters :
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

pipeline = Pipeline([
        ('selector', BaseColumnsSelector(baseColumns)),
        ('outliers_replacer', OutliersReplacer()),
        ('ratio_columns_adder', RatioColumnsAdder()),
        ('missing_values_filler', MissingValuesFiller()),
        ('drop_columns', DropColumns()),
        ('classifier', GradientBoostingClassifier(n_estimators=18, max_depth=2, learning_rate=0.2))
    ])
#validation 
kfold = KFold(n_splits=10, random_state=42, shuffle=True)
score_acc = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
score_recall = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='recall')
score_precision = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='precision')
score_f1 = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='f1')
print(f"accuracy: {score_acc.mean()}, recall: {score_recall.mean()}, precision: {score_precision.mean()}, f1: {score_f1.mean()}")



accuracy: 0.7359375, recall: 0.19678431174948202, precision: 0.5853787878787878, f1: 0.28940148448043185


In [10]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.74375
[[110   4]
 [ 37   9]]
              precision    recall  f1-score   support

           0       0.75      0.96      0.84       114
           1       0.69      0.20      0.31        46

    accuracy                           0.74       160
   macro avg       0.72      0.58      0.57       160
weighted avg       0.73      0.74      0.69       160

